# Removing all countries without Medal wins

# Purpose 
Any country that has not won a medal are no use for our study on the factors that affect success so we'll remove them. 

# Datasets
Uses: <br>
** countries_200.csv ** from 200-Country_Preparation <br>
Creates: &emsp;
<br>
** countries_250.csv ** csv containing country information on countries who have won a medal(s)

In [1]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
from bs4 import BeautifulSoup
import webbrowser
import urllib.request
from lxml import html
import zipfile
import re
import string
import sys, os
from IPython.display import Image

In [2]:
# Ensure the file exists
if not os.path.exists(  r"../../data/prep/Countries/countries_200.csv" ):
    print("Missing dataset file")

In [3]:
# read the countries df into a dataframe
df = pd.read_csv(  r"../../data/prep/Countries/countries_200.csv", encoding = "ISO-8859-1")

# Country df
This DataFrame called df has information on all the factors we'll be analysing for all the countries.  

In [4]:
# printing the country df out looking at field names 
df.columns

Index(['Country', 'Year', 'Population', 'Males', 'Females', 'Life_Expectancy',
       'GDP', 'Region', 'Elevation', 'Area_SqKM', 'Centroid_Longitude',
       'Centroid_Latitude', 'Population_Density', 'CO2_Emissions',
       'Methane_Emissions', 'Nitrous_Oxide_Emisions', 'Total_Emissions',
       'Emmisions_per_Capita'],
      dtype='object')

In [5]:
# checking the number of null values we are dealing with
df.isnull().sum()

Country                      0
Year                         0
Population                   0
Males                        0
Females                      0
Life_Expectancy            455
GDP                        892
Region                       0
Elevation                  861
Area_SqKM                  861
Centroid_Longitude         861
Centroid_Latitude          861
Population_Density         861
CO2_Emissions              675
Methane_Emissions         1118
Nitrous_Oxide_Emisions    1102
Total_Emissions              0
Emmisions_per_Capita      4158
dtype: int64

# Removing countries that have not won any medals 
We don't need any information on any countries outside of the countries who won medals. As mentioned above

In [6]:
# Ensure the file exists
if not os.path.exists(  r"../../data/raw/dictionary.csv" ):
    print("Missing dataset file")

In [7]:
# read the dictionary df into a dataframe
dfD = pd.read_csv(  r"../../data/raw/dictionary.csv", encoding = "ISO-8859-1")

# Dictionary DataFrame 
This dataFrame/ csv contains all the country names along with their country code and region. 

In [8]:
dfD.columns

Index(['Country', 'Code', 'Region'], dtype='object')

In [9]:
# Ensure the file exists
if not os.path.exists(  r"..\..\data\prep\Games\Games-950.csv" ):
    print("Missing dataset file")

In [10]:
# read the medaldf df into a dataframe
Medaldf = pd.read_csv(  r"..\..\data\prep\Games\Games-950.csv", encoding = "ISO-8859-1")

# Medal DataFrame 
The medal dataFrame contains records of all the medals won and information on the countries and athletes that won them. We'll need this dataFrame in order to find out which countries won medals then telling us which countries we can remove from the country dataFrame 'df'.

In [11]:
Medaldf.columns

Index(['Year', 'Host_Country', 'Host_City', 'Summer', 'Winter', 'Total_Males',
       'Total_Females', 'Total_Athletes', 'Discipline', 'Sport', 'Ath_Name',
       'Gender', 'Home_Adv', 'Gold', 'Silver', 'Bronze', 'Total_Medals',
       'Ath_Rating', 'Ath_Rank', 'NOC', 'NOC_Males_Sent', 'NOC_Females_Sent',
       'NOC_Total_Sent', 'NOC_Gold', 'NOC_Silver', 'NOC_Bronze',
       'NOC_Total_Medals', 'NOC_Rating', 'NOC_Rank'],
      dtype='object')

In [12]:
# These are the NOCs that matter 
Medaldf['NOC'].unique()

array(['ARG', 'AUS', 'AUT', 'BEL', 'BRA', 'BUL', 'BWI', 'CAN', 'DEN',
       'EGY', 'ESP', 'ETH', 'EUA', 'FIN', 'FRA', 'GBR', 'GHA', 'GRE',
       'HUN', 'IND', 'IRI', 'IRQ', 'ITA', 'JPN', 'MAR', 'MEX', 'NED',
       'NOR', 'NZL', 'PAK', 'POL', 'POR', 'ROU', 'RSA', 'SGP', 'SUI',
       'SWE', 'TCH', 'TPE', 'TUR', 'URS', 'USA', 'VEN', 'YUG', 'PRK',
       'BAH', 'CUB', 'IRL', 'KEN', 'KOR', 'NGR', 'PHI', 'TTO', 'TUN',
       'URU', 'FRG', 'GDR', 'CMR', 'JAM', 'MGL', 'UGA', 'COL', 'LBN',
       'NIG', 'LIE', 'BER', 'PUR', 'THA', 'GUY', 'TAN', 'ZIM', 'ALG',
       'CHN', 'CIV', 'DOM', 'ISL', 'PER', 'SYR', 'ZAM', 'AHO', 'CHI',
       'CRC', 'DJI', 'INA', 'ISV', 'SEN', 'SUR', 'CRO', 'EST', 'EUN',
       'GER', 'IOP', 'ISR', 'LAT', 'LTU', 'MAS', 'NAM', 'QAT', 'SLO',
       'LUX', 'BLR', 'KAZ', 'RUS', 'UKR', 'UZB', 'ARM', 'AZE', 'BDI',
       'CZE', 'ECU', 'GEO', 'HKG', 'MDA', 'MOZ', 'SVK', 'TGA', 'BAR',
       'KGZ', 'KSA', 'KUW', 'MKD', 'SRI', 'VIE', 'ERI', 'PAR', 'SCG',
       'UAE', 'AFG',

# Getting the NOCs of each country in the Country df
The country dataFrame we have does not currently have the country code for each country so we'll use the dictionary df to create and populate this field. To populate the code field I'll create a dictionary containing all the countries as keys and their NOCs as values. 

In [13]:
countrydf = df

In [14]:
countrydf.columns

Index(['Country', 'Year', 'Population', 'Males', 'Females', 'Life_Expectancy',
       'GDP', 'Region', 'Elevation', 'Area_SqKM', 'Centroid_Longitude',
       'Centroid_Latitude', 'Population_Density', 'CO2_Emissions',
       'Methane_Emissions', 'Nitrous_Oxide_Emisions', 'Total_Emissions',
       'Emmisions_per_Capita'],
      dtype='object')

In [15]:
# creating the dictionary 
countrydic = {}

# Creating the dictionary of countries and country codes 

In [16]:
# For that fills the country name, NOC dictionary 
dfD['Dic'] = None 

for x, row in dfD.iterrows():
    
    country = dfD['Country'].iloc[x]
    code = dfD['Code'].iloc[x]
    
    countrydic.update({country: code})

# A look at the Country Dictionary 

In [17]:
countrydic = {'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Algeria': 'ALG',
 'American Samoa': 'ASA',
 'Andorra': 'AND',
 'Angola': 'ANG',
 'Antigua and Barbuda': 'ANT',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Aruba': 'ARU',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Bahamas': 'BAH',
 'Bahamas, The' : 'BAH',       
 'Bahrain': 'BRN',
 'Bangladesh': 'BAN',
 'Barbados': 'BAR',
 'Belarus': 'BLR',
 'Belgium': 'BEL',
 'Belize': 'BIZ',
 'Benin': 'BEN',
 'Bermuda': 'BER',
 'Bhutan': 'BHU',
 'Bolivia': 'BOL',
 'Bosnia and Herzegovina': 'BIH',
 'Botswana': 'BOT',
 'Brazil': 'BRA',
 'British Virgin Islands': 'IVB',
 'Brunei Darussalam': 'BRU',
 'Bulgaria': 'BUL',
 'Burkina Faso': 'BUR',
 'Burundi': 'BDI',
 'Cabo Verde': 'CPV',
 'Cambodia': 'CAM',
 'Cameroon': 'CMR',
 'Canada': 'CAN',
 'Cayman Islands': 'CAY',
 'Central African Republic': 'CAF',
 'Chad': 'CHA',
 'Chile': 'CHI',
 'China': 'CHN',
 'Colombia': 'COL',
 'Comoros': 'COM',
 'Congo, Dem Rep': 'COD', 
 'Congo, Dem. Rep.': 'COD', 
 'Costa Rica': 'CRC',
 "Cote d'Ivoire": 'CIV',
 'Croatia': 'CRO',
 'Cuba': 'CUB',
 'Curacao': 'CUW',
 'Cyprus': 'CYP',
 'Czech Republic': 'CZE',
 'Denmark': 'DEN',
 'Djibouti': 'DJI',
 'Dominica': 'DMA',
 'Dominican Republic': 'DOM',
 'Ecuador': 'ECU',
 'Egypt Arab Rep': 'EGY',
 'Egypt, Arab Rep.': 'EGY',
 'El Salvador': 'ESA',
 'Equatorial Guinea': 'GEQ',
 'Eritrea': 'ERI',
 'Estonia': 'EST',
 'Ethiopia': 'ETH',
 'Fiji': 'FIJ',
 'Finland': 'FIN',
 'France': 'FRA',
 'Gabon': 'GAB',
 'Gambia': 'GAM',
 'Gambia, The': 'GAM',
 'Georgia': 'GEO',
 'Germany': 'GER',
 'Ghana': 'GHA',
 'Greece': 'GRE',
 'Grenada': 'GRN',
 'Guam': 'GUM',
 'Guatemala': 'GUA',
 'Guinea': 'GUI',
 'Guinea-Bissau': 'GBS',
 'Guyana': 'GUY',
 'Haiti': 'HAI',
 'Honduras': 'HON',
 'Hong Kong SAR, China': 'HKG',
 'Hungary': 'HUN',
 'Iceland': 'ISL',
 'India': 'IND',
 'Indonesia': 'INA',
 'Iran, Islamic Rep.': 'IRI',
 'Iraq': 'IRQ',
 'Ireland': 'IRL',
 'Israel': 'ISR',
 'Italy': 'ITA',
 'Jamaica': 'JAM',
 'Japan': 'JPN',
 'Jordan': 'JOR',
 'Kazakhstan': 'KAZ',
 'Kenya': 'KEN',
 "Korea, Dem. People\x92s Rep.": 'PRK',
 'Korea, Dem. People?s Rep.': 'PRK',
 'Korea, Rep.': 'KOR',
 'Kosovo': 'KOS',
 'Kuwait': 'KUW',
 'Kyrgyz Republic': 'KGZ',
 'Lao PDR': 'LAO',
 'Latvia': 'LAT',
 'Lebanon': 'LIB',
 'Lesotho': 'LES',
 'Liberia': 'LBR',
 'Libya': 'LBA',
 'Liechtenstein': 'LIE',
 'Lithuania': 'LTU',
 'Luxembourg': 'LUX',
 'Macedonia, FYR': 'MKD',
 'Madagascar': 'MAD',
 'Malawi': 'MAW',
 'Malaysia': 'MAS',
 'Maldives': 'MDV',
 'Mali': 'MLI',
 'Malta': 'MLT',
 'Mauritania': 'MTN',
 'Mauritius': 'MRI',
 'Mexico': 'MEX',
 'Micronesia Fed. Sts.': 'FSM',
 'Micronesia, Fed. Sts.': 'FSM', 
 'Moldova': 'MDA',
 'Monaco': 'MON',
 'Mongolia': 'MGL',
 'Montenegro': 'MNE',
 'Morocco': 'MAR',
 'Mozambique': 'MOZ',
 'Myanmar': 'MYA',
 'Namibia': 'NAM',
 'Nauru': 'NRU',
 'Nepal': 'NEP',
 'Netherlands': 'NED',
 'New Zealand': 'NZL',
 'Nicaragua': 'NCA',
 'Niger': 'NIG',
 'Nigeria': 'NGR',
 'Norway': 'NOR',
 'Oman': 'OMA',
 'Pakistan': 'PAK',
 'Palau': 'PLW',
 'Panama': 'PAN',
 'Papua New Guinea': 'PNG',
 'Paraguay': 'PAR',
 'Peru': 'PER',
 'Philippines': 'PHI',
 'Poland': 'POL',
 'Portugal': 'POR',
 'Puerto Rico': 'PUR',
 'Qatar': 'QAT',
 'Romania': 'ROU',
 'Russian Federation': 'RUS',
 'Rwanda': 'RWA',
 'Samoa': 'SAM',
 'San Marino': 'SMR',
 'Sao Tome and Principe': 'STP',
 'Saudi Arabia': 'KSA',
 'Senegal': 'SEN',
 'Serbia': 'SRB',
 'Seychelles': 'SEY',
 'Sierra Leone': 'SLE',
 'Singapore': 'SGP',
 'Slovak Republic': 'SVK',
 'Slovenia': 'SLO',
 'Solomon Islands': 'SOL',
 'Somalia': 'SOM',
 'South Africa': 'RSA',
 'Spain': 'ESP',
 'Sri Lanka': 'SRI',
 'St. Kitts and Nevis': 'SKN',
 'St. Lucia': 'LCA',
 'St. Vincent and the Grenadines': 'VIN',
 'Sudan': 'SUD',
 'Suriname': 'SUR',
 'Swaziland': 'SWZ',
 'Sweden': 'SWE',
 'Switzerland': 'SUI',
 'Syrian Arab Republic': 'SYR',
 'Tajikistan': 'TJK',
 'Tanzania': 'TAN',
 'Thailand': 'THA',
 'Timor-Leste': 'TLS',
 'Togo': 'TOG',
 'Tonga': 'TGA',
 'Trinidad and Tobago': 'TTO',
 'Tunisia': 'TUN',
 'Turkey': 'TUR',
 'Turkmenistan': 'TKM',
 'Uganda': 'UGA',
 'Ukraine': 'UKR',
 'United Arab Emirates': 'UAE',
 'United Kingdom': 'GBR',
 'United States': 'USA',
 'Uruguay': 'URU',
 'Uzbekistan': 'UZB',
 'Vanuatu': 'VAN',
 'Venezuela': 'VEN',
 'Venezuela, RB' : 'VEN',
 'Vietnam': 'VIE',
 'Virgin Islands (US)': 'ISV',
 'Virgin Islands (U.S.)': 'ISV',
 'Yemen, Rep.': 'YEM',
 'Zambia': 'ZAM',
 'Zimbabwe': 'ZIM'}

# Populating the Code Field
We'll  iterate through the country df and enter the country name 'key' into the dictionary getting the value 'code' back.

In [18]:
# adding all the NOCs from the dictionary into the countrydf 
countrydf['Code'] = None 

for x, row in countrydf.iterrows():
    
    country = countrydf['Country'].iloc[x]
    countrydf.loc[x, 'Code'] = countrydic[country]

# Countries that change whose NOCs change between 1960 - 2016
The are a few cases where countires like Germany and Soviet Union split and different countries within there bounds end up completing for different teams like the Unified Olympic Team. 
We'll will not address the issues of countries playing for different countries in this notebook for now we will just leave them in. 
The list below represents the countires that are invloved in cases where NOCs switch. 

* All these team switch NOCs or join another at one point.
* We'll get the unique conturies and leave them in the country df and get values for their factors then in another notebook we can join them later to their respective NOCs. 

In [19]:
members = ['Armenia','Azerbaijan','Belarus','Georgia','Kazakhstan','Kyrgyz Republic','Moldova',
           'Russian Federation','Tajikistan','Turkmenistan','Ukraine','Uzbekistan','Latvia', 
            'Armenia','Belarus','Kazakhstan','Russian Federation','Uzbekistan',
            'Serbia','Montenegro','Macedonia, FYR','Bosnia and Herzegovina','Slovenia','Croatia', 
            'Armenia', 'Belarus', 'Georgia', 'Kazakhstan', 'Kyrgyz Republic', 
            'Moldova','Russian Federation', 'Ukraine', 'Uzbekistan', 'Czech Republic','Slovak Republic', 'Serbia','Montenegro', 
           'Estonia', 'Lativa', 'Lithuania']

In [20]:
# now we can get the unique countries because some repeat in this list
# We'll do this by changing it to a set then back to a list 
otherCslist = list(set(members))

In [21]:
# create a df for other countires above so we can get their country codes, by joining to the dictionary df  
otherCsdf = pd.DataFrame(columns=['Country'])
otherCsdf['Country'] = otherCslist

In [22]:
# now we can join this to the dictonary and get all its Country codes 
otherCsdf.columns

Index(['Country'], dtype='object')

In [23]:
len(otherCsdf)

24

In [24]:
# creating a temp dictionary so we can get the country codes for only the countries in this otherCsList
tempDict = dfD
# then we are removing all the rows with countries that are not in the otherCslist and creating a df
# this df contains all the countries and country codes of the country that change NOcs that we need to keep with the Medaldf
otherCsdf = tempDict[tempDict['Country'].isin(otherCslist)].reset_index()
# Now we can remove the old index field because it is redudant 
otherCsdf = otherCsdf.drop(otherCsdf.columns[[0]], axis=1)

In [25]:
# All the country codes for all the coutries that change NOCs
otherCsdf['Code'].unique()

array(['ARM', 'AZE', 'BLR', 'BIH', 'CRO', 'CZE', 'EST', 'GEO', 'KAZ',
       'KGZ', 'LAT', 'LTU', 'MKD', 'MDA', 'RUS', 'SRB', 'SVK', 'SLO',
       'TJK', 'TKM', 'UKR', 'UZB', 'MNE'], dtype=object)

In [26]:
# Making the country codes above into a list 
otherCslist = ['ARM', 'AZE', 'BLR', 'BIH', 'CRO', 'CZE', 'GEO', 'KAZ', 'KGZ',
       'LAT', 'MKD', 'MDA', 'RUS', 'SRB', 'SVK', 'SLO', 'TJK', 'TKM',
       'UKR', 'UZB', 'MNE']

# Adding the two lists so we'll have a list of only the countries we need for the final country df
So we'll add this otherCslist to the unique Nocs in the medalDf and we'll have a list all of these countires 

In [27]:
# All the country codes for the counries who have won Medals 
Medaldf['NOC'].unique()

array(['ARG', 'AUS', 'AUT', 'BEL', 'BRA', 'BUL', 'BWI', 'CAN', 'DEN',
       'EGY', 'ESP', 'ETH', 'EUA', 'FIN', 'FRA', 'GBR', 'GHA', 'GRE',
       'HUN', 'IND', 'IRI', 'IRQ', 'ITA', 'JPN', 'MAR', 'MEX', 'NED',
       'NOR', 'NZL', 'PAK', 'POL', 'POR', 'ROU', 'RSA', 'SGP', 'SUI',
       'SWE', 'TCH', 'TPE', 'TUR', 'URS', 'USA', 'VEN', 'YUG', 'PRK',
       'BAH', 'CUB', 'IRL', 'KEN', 'KOR', 'NGR', 'PHI', 'TTO', 'TUN',
       'URU', 'FRG', 'GDR', 'CMR', 'JAM', 'MGL', 'UGA', 'COL', 'LBN',
       'NIG', 'LIE', 'BER', 'PUR', 'THA', 'GUY', 'TAN', 'ZIM', 'ALG',
       'CHN', 'CIV', 'DOM', 'ISL', 'PER', 'SYR', 'ZAM', 'AHO', 'CHI',
       'CRC', 'DJI', 'INA', 'ISV', 'SEN', 'SUR', 'CRO', 'EST', 'EUN',
       'GER', 'IOP', 'ISR', 'LAT', 'LTU', 'MAS', 'NAM', 'QAT', 'SLO',
       'LUX', 'BLR', 'KAZ', 'RUS', 'UKR', 'UZB', 'ARM', 'AZE', 'BDI',
       'CZE', 'ECU', 'GEO', 'HKG', 'MDA', 'MOZ', 'SVK', 'TGA', 'BAR',
       'KGZ', 'KSA', 'KUW', 'MKD', 'SRI', 'VIE', 'ERI', 'PAR', 'SCG',
       'UAE', 'AFG',

In [28]:
# turning these country codes for the winners into a list 
MedaldfCs = ['ARG', 'AUS', 'AUT', 'BEL', 'BRA', 'BUL', 'BWI', 'CAN', 'DEN',
       'EGY', 'ESP', 'ETH', 'EUA', 'FIN', 'FRA', 'GBR', 'GHA', 'GRE',
       'HUN', 'IND', 'IRI', 'IRQ', 'ITA', 'JPN', 'MAR', 'MEX', 'NED',
       'NOR', 'NZL', 'PAK', 'POL', 'POR', 'ROU', 'RSA', 'SGP', 'SUI',
       'SWE', 'TCH', 'TPE', 'TUR', 'URS', 'USA', 'VEN', 'YUG', 'PRK',
       'BAH', 'CUB', 'IRL', 'KEN', 'KOR', 'NGR', 'PHI', 'TTO', 'TUN',
       'URU', 'FRG', 'GDR', 'CMR', 'JAM', 'MGL', 'UGA', 'COL', 'LBN',
       'NIG', 'LIE', 'BER', 'PUR', 'THA', 'GUY', 'TAN', 'ZIM', 'ALG',
       'CHN', 'CIV', 'DOM', 'ISL', 'PER', 'SYR', 'ZAM', 'AHO', 'CHI',
       'CRC', 'DJI', 'INA', 'ISV', 'SEN', 'SUR', 'CRO', 'EST', 'EUN',
       'GER', 'IOP', 'ISR', 'LAT', 'LTU', 'MAS', 'NAM', 'QAT', 'SLO',
       'LUX', 'BLR', 'KAZ', 'RUS', 'UKR', 'UZB', 'ARM', 'AZE', 'BDI',
       'CZE', 'ECU', 'GEO', 'HKG', 'MDA', 'MOZ', 'SVK', 'TGA', 'BAR',
       'KGZ', 'KSA', 'KUW', 'MKD', 'SRI', 'VIE', 'ERI', 'PAR', 'SCG',
       'UAE', 'AFG', 'MRI', 'PAN', 'SAM', 'SRB', 'SUD', 'TJK', 'TOG',
       'BOT', 'BRN', 'CYP', 'GAB', 'GRN', 'GUA', 'MNE', 'FIJ', 'IOA',
       'JOR', 'KOS', 'OAR']

In [29]:
# this is a list of the relevant countries 
# the winners country codes and the country codes from the countries who have changed NOCs from 1960 - 2016
RelCs = list(set(otherCslist + MedaldfCs))

# Finally removing all the irrelevant countries from the country df
We'll remove all the rows from the country df that are not in this RelCs list above

In [30]:
countrydf = countrydf[countrydf['Code'].isin(RelCs)].reset_index()
# Now we can remove the old index field because it is redudant 
countrydf = countrydf.drop(countrydf.columns[[0]], axis=1)

# This country dataFrame now only contains the relevant countires who have won medals

In [31]:
df = countrydf

In [32]:
df.to_csv( r"../../data/prep/Countries/countries_250.csv", index=False)